In [26]:
%reset -f
import pandas as pd
import numpy as np
from wavhandler import *
import soundfile as sf
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline

ERROR:root:Invalid alias: The name clear can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name more can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name less can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name man can't be aliased because it is another magic command.


In [27]:
model_name = 'raw_find_Culex'
top_weights_path = TEMP_DATADIR + '/model_' + str(model_name) + '.h5'
logfile = TEMP_DATADIR + 'model_' + str(model_name) + '.log'

In [28]:
# data1 = Dataset('Wingbeats')
# data1.load(only_names=True, nr_signals=5000, text_labels=True);
# data2 = Dataset('LG')
# data2.load(only_names=True, text_labels=True);
# data_X_names = data1.filenames + data2.filenames
# data_y = data1.y + data2.y

In [29]:
data1 = Dataset('Wingbeats')
data1.target_classes = data1.target_classes[:4]
data1.load(only_names=True, nr_signals=5000, text_labels=True)
data2 = Dataset('LG')
data2.load(only_names=True, text_labels=True);
data_X_names = data1.filenames + data2.filenames
data_y = data1.y + data2.y
print(np.unique(data_y))

100%|██████████| 2/2 [00:00<00:00, 36.66it/s]

['Ae. aegypti' 'Ae. albopictus' 'An. arabiensis' 'An. gambiae'
 'LG_drosophila_10_09' 'LG_zapr_26_09']


In [30]:
%%time
df = make_df_parallel(names=data_X_names, setting='read').T

CPU times: user 5.61 s, sys: 1.78 s, total: 7.39 s
Wall time: 8.11 s


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

y = LabelEncoder().fit_transform(y=data_y)

X_train, X_test, y_train, y_test = train_test_split(df.values, y, 
                                                    test_size=0.10, 
                                                    shuffle=True, 
                                                    random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, 
                                                  test_size=0.2, 
                                                  random_state=0)
print("Train shape: \t{}, \nTest shape: \t{}, \nValid shape: \t{}".format(X_train.shape, X_test.shape, X_val.shape))

In [ ]:
import keras

nr_classes = len(np.unique(data_y))

# Convert label to onehot
y_train = keras.utils.to_categorical(y_train, num_classes=nr_classes)
y_val = keras.utils.to_categorical(y_val, num_classes=nr_classes)
y_test = keras.utils.to_categorical(y_test, num_classes=nr_classes)

X_train = np.expand_dims(X_train, axis=2)
X_val = np.expand_dims(X_val, axis=2)
X_test = np.expand_dims(X_test, axis=2)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D
from keras.optimizers import SGD
from keras.layers.normalization import BatchNormalization
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger

# Build the Neural Network
model = Sequential()

model.add(Conv1D(16, 3, activation='relu', input_shape=(5000, 1)))
model.add(Conv1D(16, 3, activation='relu'))
model.add(BatchNormalization())

model.add(Conv1D(32, 3, activation='relu'))
model.add(Conv1D(32, 3, activation='relu'))
model.add(BatchNormalization())

model.add(MaxPooling1D(2))
model.add(Conv1D(64, 3, activation='relu'))
model.add(Conv1D(64, 3, activation='relu'))
model.add(BatchNormalization())

model.add(MaxPooling1D(2))
model.add(Conv1D(128, 3, activation='relu'))
model.add(Conv1D(128, 3, activation='relu'))
model.add(BatchNormalization())

model.add(MaxPooling1D(2))
model.add(Conv1D(256, 3, activation='relu'))
model.add(Conv1D(256, 3, activation='relu'))
model.add(BatchNormalization())
model.add(GlobalAveragePooling1D())

model.add(Dropout(0.5))
model.add(Dense(nr_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
callbacks_list = [ModelCheckpoint(top_weights_path, monitor = 'val_acc', verbose = 1, save_best_only = True, save_weights_only = True),
    EarlyStopping(monitor = 'val_acc', patience = 6, verbose = 1),
    ReduceLROnPlateau(monitor = 'val_acc', factor = 0.1, patience = 3, verbose = 1),
    CSVLogger(logfile)]

In [ ]:
model.fit(X_train, y_train, batch_size=32, epochs=100, validation_data = [X_val, y_val], callbacks = callbacks_list)


In [ ]:
model.load_weights(top_weights_path)
loss, acc = model.evaluate(X_test, y_test, batch_size=16)

#print('loss', loss)
print('Test accuracy:', acc)

In [ ]:
from keras.models import model_from_yaml
# serialize model to YAML
model_yaml = model.to_yaml()
with open(TEMP_DATADIR + model_name + ".yaml", "w") as yaml_file:
    yaml_file.write(model_yaml)
model.save_weights(TEMP_DATADIR + model_name + "_weights.h5")

In [2]:
%reset -f
from utils import *
from keras.models import model_from_yaml
from wavhandler import read_simple
from wavhandler import *
import keras
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

model_name = 'raw_find_Culex'
yaml_file = open(TEMP_DATADIR + model_name + '.yaml', 'r')
model = yaml_file.read()
yaml_file.close()
model = model_from_yaml(model)
model.load_weights(TEMP_DATADIR + model_name + '_weights.h5')

ERROR:root:Invalid alias: The name clear can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name more can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name less can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name man can't be aliased because it is another magic command.
Using TensorFlow backend.


In [3]:
from keras.models import Model
layer_name = model.layers[-1].name
print("Loading weights from layer: {}".format(layer_name))

# out = Model(inputs=model.input, outputs=model.get_layer(layer_name).output)
# out = keras.layers.Dense(2, activation='softmax')(out.output)
intermediate_layer_model = Model(inputs=model.input, outputs=model.get_layer(layer_name).output)

Loading weights from layer: dense_1


In [4]:
data = Dataset('Wingbeats')
data.target_classes = [i for i in data.target_classes if i.startswith('An.')]
data.load(only_names=True, nr_signals=10000, text_labels=True)

df = pd.DataFrame({'filenames':data.filenames, 'y': data.y})
df.head()

#df = data_X_names.get_sensor_features(temp_humd=False)
data = read_simple(df.filenames)[0].T
data = np.expand_dims(data, axis=-1)
data.shape

100%|██████████| 2/2 [00:00<00:00, 11.09it/s]


(20489, 5000, 1)

In [5]:
d = intermediate_layer_model.predict(data)
d.shape

UnknownError: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node conv1d_1/convolution/Conv2D}} = Conv2D[T=DT_FLOAT, data_format="NCHW", dilations=[1, 1, 1, 1], padding="VALID", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](conv1d_1/convolution/Conv2D-0-TransposeNHWCToNCHW-LayoutOptimizer, conv1d_1/convolution/ExpandDims_1)]]

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=2, random_state=0).fit(d)

In [ ]:
from sklearn.preprocessing import LabelEncoder

LabelEncoder().fit_transform(df.y)

In [ ]:
plt.plot(df.y)

In [ ]:
# yday = (df.date_day <= zz).astype(int)
# hue = {'day': yday, 'deep': }
hue = kmeans.labels_
hue = df.y

X = d
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

X_std = StandardScaler(with_std=True).fit_transform(X);  # standardization of data
ncomps = 3
pca = PCA(n_components=ncomps)
X_final = pca.fit_transform(X_std)

print("EXPLAINED VARIANCE with {0:d} components: {1:.2f}".format(ncomps,pca.explained_variance_ratio_.sum()) )
plt.figure(figsize=(10,8))
sns.set_style("darkgrid", {'axes.grid' : False})
sns.scatterplot(X_final[:,0], X_final[:,1],alpha=0.8, legend='full', palette=sns.color_palette("cubehelix", len(np.unique(hue))), hue=hue)#df.label)#yday)#
plt.xlabel('component 1')
plt.ylabel('component 2')

In [ ]:
import plotly
import plotly.plotly as py
from sklearn.preprocessing import LabelEncoder
plotly.tools.set_credentials_file(username='kalfasyan', api_key='oOr9ILFmszXQYREloJTp')

import plotly.graph_objs as go

import numpy as np

x, y, z = X_final[:,0], X_final[:,1], X_final[:,2]
trace1 = go.Scatter3d(
    x=x,
    y=y,
    z=z,
    mode='markers',
    marker=dict(
        size=4,
        line=dict(
            color='rgba(217, 217, 217, 0.14)',#LabelEncoder().fit(target_names).transform(df_concat.label.tolist()),#
            colorscale='Portland',
            width=0.5
        ),
        opacity=0.9
    )
)


data = [trace1]
layout = go.Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='simple-3d-scatter')